# Reading files and working with JSON

In [80]:
import json
import os
import pandas as pd
import urllib

In [12]:
!ls -R data

log_data  song_data

data/log_data:
2018

data/log_data/2018:
11

data/log_data/2018/11:
2018-11-01-events.json 2018-11-11-events.json 2018-11-21-events.json
2018-11-02-events.json 2018-11-12-events.json 2018-11-22-events.json
2018-11-03-events.json 2018-11-13-events.json 2018-11-23-events.json
2018-11-04-events.json 2018-11-14-events.json 2018-11-24-events.json
2018-11-05-events.json 2018-11-15-events.json 2018-11-25-events.json
2018-11-06-events.json 2018-11-16-events.json 2018-11-26-events.json
2018-11-07-events.json 2018-11-17-events.json 2018-11-27-events.json
2018-11-08-events.json 2018-11-18-events.json 2018-11-28-events.json
2018-11-09-events.json 2018-11-19-events.json 2018-11-29-events.json
2018-11-10-events.json 2018-11-20-events.json 2018-11-30-events.json

data/song_data:
A

data/song_data/A:
A B

data/song_data/A/A:
A B C

data/song_data/A/A/A:
TRAAAAW128F429D538.json TRAAAFD128F92F423A.json TRAAARJ128F9320760.json
TRAAABD128F429CF47.json TRAAAMO128F1481E7F.json TRAAAVG12

In [43]:
SONG_PATH = os.path.join('data', 'song_data', 'A')
LOG_PATH = os.path.join('data', 'log_data')
LOG_NOV2018 = os.path.join(LOG_PATH, '2018', '11')

In [44]:
print(repr(SONG_PATH))

'data/song_data/A'


In [33]:
log_file_paths = [f.path for f in os.scandir(LOG_NOV2018) if f.is_file()]

Note: `scandir` is an iterator of `os.DirEntry` objects, which have a `.is_file()` method; there is also a method `os.path.isfile(some_path)` that checks if `some_path` is a file.

In [37]:
print(repr(log_file_paths[0]))

'data/log_data/2018/11/2018-11-11-events.json'


In [39]:
def get_json(path):
    assert path[-5:] == '.json'
    with open(path, 'r') as f:
        return json.load(f)


log0 = get_json(log_file_paths[0])
print(log0)

JSONDecodeError: Extra data: line 2 column 1 (char 496)

[jsonlint.com] says that the JSON log files aren't formatted correctly.

## Reading song files

Let's try working on loading from the song files, which seem to be valid JSON, then try to fix the log files.

In [46]:
song_file_paths = []
for x in ['A', 'B']:
    for y in ['A', 'B', 'C']:
        temp_path = os.path.join(SONG_PATH, x, y)
        new_paths = [f.path for f in os.scandir(temp_path) if f.is_file()]
        song_file_paths += new_paths
    
print(repr(song_file_paths[0]))

'data/song_data/A/A/A/TRAAAEF128F4273421.json'


In [47]:
song0 = get_json(song_file_paths[0])
print(repr(song0))

{'num_songs': 1, 'artist_id': 'AR7G5I41187FB4CE6C', 'artist_latitude': None, 'artist_longitude': None, 'artist_location': 'London, England', 'artist_name': 'Adam Ant', 'song_id': 'SONHOTT12A8C13493C', 'title': 'Something Girls', 'duration': 233.40363, 'year': 1982}


In [130]:
song0_df = pd.DataFrame([song0])
song0_df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,None,None,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


In [131]:
song0_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num_songs         1 non-null      int64  
 1   artist_id         1 non-null      object 
 2   artist_latitude   0 non-null      object 
 3   artist_longitude  0 non-null      object 
 4   artist_location   1 non-null      object 
 5   artist_name       1 non-null      object 
 6   song_id           1 non-null      object 
 7   title             1 non-null      object 
 8   duration          1 non-null      float64
 9   year              1 non-null      int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 208.0+ bytes


In [51]:
song00 = pd.read_json(song_file_paths[0], typ='series')
print(song00)

num_songs                            1
artist_id           AR7G5I41187FB4CE6C
artist_latitude                   None
artist_longitude                  None
artist_location        London, England
artist_name                   Adam Ant
song_id             SONHOTT12A8C13493C
title                  Something Girls
duration                       233.404
year                              1982
dtype: object


In [54]:
type(song00)

pandas.core.series.Series

Note: without the option `typ='series'`, we get an error, since the JSON files are not arrays (so there is a problem with indexing). Thus we might need to cast to DataFrame later.

Another option might be to set `lines = True`; let's try this.

In [62]:
song000 = pd.read_json(song_file_paths[0], lines=True)
song000

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


This works as well.

## Handling multiple files

How can I import all of the files in a directory to a single DataFrame?

- One option is to create a list of DataFrames, then concatenate them.
- OTOH, if we want to import the JSON files one at a time, we could probably just loop through. It depends if we need to cache, and if DataFrames are a convenient way to do this

# Fixing the log files

The log files don't seem to be valid JSON... let's double check with JSONlint.com
The log JSONs seem to have multiple dictionaries/JSON objects without proper delimiters between them.

## Attempt 1

Let's try to put these log entries into an array.

We need to:
- read in the log files
- split out "{.*}"
- put these in a list, say
- joint on ',' and add square brackets
- then we should be able to read via `json.loads`

Issue: I don't know what the delimiters are in the log JSONs. It looks like '\n' when I open it in a text editor.

In [58]:
with open(log_file_paths[0], 'r') as f:
    for i, line in enumerate(f, 1):
        print(f'line {i}:\n{line}\n')

line 1:
{"artist":"Frumpies","auth":"Logged In","firstName":"Anabelle","gender":"F","itemInSession":0,"lastName":"Simpson","length":134.47791,"level":"free","location":"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD","method":"PUT","page":"NextSong","registration":1541044398796.0,"sessionId":455,"song":"Fuck Kitty","status":200,"ts":1541903636796,"userAgent":"\"Mozilla\/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit\/537.36 (KHTML, like Gecko) Chrome\/36.0.1985.125 Safari\/537.36\"","userId":"69"}


line 2:
{"artist":"Kenny G with Peabo Bryson","auth":"Logged In","firstName":"Anabelle","gender":"F","itemInSession":1,"lastName":"Simpson","length":264.75057,"level":"free","location":"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD","method":"PUT","page":"NextSong","registration":1541044398796.0,"sessionId":455,"song":"By The Time This Night Is Over","status":200,"ts":1541903770796,"userAgent":"\"Mozilla\/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit\/537.36 (KHTML, like Gecko) Chrome\/

So the delimiters are newlines. Now let's return a list of the lines.

In [60]:
def get_log_lines(path):
    with open(path, 'r') as f:
        return [line for line in f]
    
log0 = get_log_lines(log_file_paths[0])
for i in range(2):
    print(log0[i])

{"artist":"Frumpies","auth":"Logged In","firstName":"Anabelle","gender":"F","itemInSession":0,"lastName":"Simpson","length":134.47791,"level":"free","location":"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD","method":"PUT","page":"NextSong","registration":1541044398796.0,"sessionId":455,"song":"Fuck Kitty","status":200,"ts":1541903636796,"userAgent":"\"Mozilla\/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit\/537.36 (KHTML, like Gecko) Chrome\/36.0.1985.125 Safari\/537.36\"","userId":"69"}

{"artist":"Kenny G with Peabo Bryson","auth":"Logged In","firstName":"Anabelle","gender":"F","itemInSession":1,"lastName":"Simpson","length":264.75057,"level":"free","location":"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD","method":"PUT","page":"NextSong","registration":1541044398796.0,"sessionId":455,"song":"By The Time This Night Is Over","status":200,"ts":1541903770796,"userAgent":"\"Mozilla\/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit\/537.36 (KHTML, like Gecko) Chrome\/36.0.1985.125 Saf

In [66]:
log00 = pd.read_json('[' + ','.join(log0) + ']')
log00.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


Let's write this as one function.

In [67]:
def log_to_df(path):
    with open(path, 'r') as f:
        temp = [line for line in f]
    return pd.read_json('[' + ','.join(temp) + ']')

log_to_df(log_file_paths[0]).head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


We could apply this method to the song files if we concatenate the files first...

- Are there issues if we have too many files to fit into memory?
- Are we given a batch of data, or is it streaming? ...batch I suppose in this case.

actually we just need to store the results of reading the files, then do the same return as `log_to_df`

Note: "Kenny G with Peabo Bryson" isn't really "atomic"... maybe we need to handle secondary artists

## Attempt 2

Apparently, the problems I was having with log_files can be fixed easily using Pandas: https://stackoverflow.com/questions/30088006/loading-a-file-with-more-than-one-line-of-json-into-pandas

The fix is to use the option `lines=True`, which accepts JSON objects separated by newlines, rather than an array of JSON object.s

In [70]:
log0000 = pd.read_json(log_file_paths[0], lines=True)
log0000.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## Loading multiple song_files into a single dataframe

Best practice is to store a list of dataframes by then concatenate them at the end, since appending to a dataframe creates a new copy in memory.

In [71]:
song1 = pd.read_json(song_file_paths[0], lines=True)
song1

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


In [75]:
def json_to_df(file_paths):
    """
    Returns a data frame containing all json files in file_paths
    
    file_paths is a list of paths to .json files
    """
    return pd.concat([pd.read_json(path, lines=True) for path in file_paths])

In [76]:
songs = json_to_df(song_file_paths)
songs.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
0,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
0,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0


Some improvements to make:
- fix index
    - use song_id?
    - what about for log files?
- for log files, fix timestamps
- need to check that json files in `file_paths` have the same format
- specify dtypes?

Note: we can run `json_to_df` with the log files as well, since they're correctly processed by `read_json` with `lines=True`.

# Moving from DataFrames to PostgreSQL

- Pandas has a to_sql feature
- Since PostgreSQL supports `COPY`, there is a faster method
- Alternatively, we could convert DataFrame --> text or CSV, then COPY directly
- DataFrames seem convenient for converting data, but I wonder if there is a more direct route. It might be possible to use `COPY` directly to create a rough DB/Schema in postgreSQL, then refine it

Excellent blog post: https://hakibenita.com/fast-load-data-python-postgresql

## Method 1: loops

In the etl Jupyter notebook, there is a loop over rows in a DataFrame that uses insert statements with placeholders `%s` and values selected from the DataFrame.

This should work with any type of SQL database.

## Method 2: write to CSV and use COPY

We can write to csv to STDIN using Pandas and Python IO, then use PostgreSQL's COPY method to write to a table from STDIN

# Testing

We need some data and databases to test our functions with.

Let's create a test database, then convert some DataFrames to json, and test the output.

In [77]:
!psql -U brendan -c "CREATE DATABASE sandbox;"

CREATE DATABASE


We can get fake JSON data from JSONPlaceholder: https://realpython.com/python-json/, see "A Real World Example (sort of)"

In [83]:
!conda install --yes -c anaconda requests

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3/envs/data_engineer_nd

  added / updated specs:
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotlipy-0.7.0             |py38haf1e3a3_1000         357 KB  anaconda
    ca-certificates-2020.10.14 |                0         127 KB  anaconda
    chardet-3.0.4              |        py38_1003         170 KB  anaconda
    cryptography-3.1.1         |   py38hddc9c9b_0         604 KB  anaconda
    idna-2.10                  |             py_0          56 KB  anaconda
    openssl-1.1.1h             |       haf1e3a3_0         3.4 MB  anaconda
    pyopenssl-19.1.0           |             py_1          47 KB  anaconda
    pysocks-1.7.1              |           py38_1          27 KB  anaconda
    requests-2.24.0            |             py_0          54 KB  anaconda
    urllib3-1.25.11           

In [84]:
import requests

In [85]:
response = requests.get("https://jsonplaceholder.typicode.com/todos")
todos = json.loads(response.text)
todos[:5]

[{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False},
 {'userId': 1,
  'id': 2,
  'title': 'quis ut nam facilis et officia qui',
  'completed': False},
 {'userId': 1, 'id': 3, 'title': 'fugiat veniam minus', 'completed': False},
 {'userId': 1, 'id': 4, 'title': 'et porro tempora', 'completed': True},
 {'userId': 1,
  'id': 5,
  'title': 'laboriosam mollitia et enim quasi adipisci quia provident illum',
  'completed': False}]

In [87]:
todos_df = pd.read_json(response.text)
todos_df.head()

,userId,id,title,completed
0,1,1,delectus aut autem,False
1,1,2,quis ut nam facilis et officia qui,False
2,1,3,fugiat veniam minus,False
3,1,4,et porro tempora,True
4,1,5,laboriosam mollitia et enim quasi adipisci qui...,False


### Pandas aside...

In [89]:
todos_df2 = pd.read_json(response.text, orient='records')
todos_df2.head()

,userId,id,title,completed
0,1,1,delectus aut autem,False
1,1,2,quis ut nam facilis et officia qui,False
2,1,3,fugiat veniam minus,False
3,1,4,et porro tempora,True
4,1,5,laboriosam mollitia et enim quasi adipisci qui...,False


In [90]:
todos_df.head(10)

,userId,id,title,completed
0,1,1,delectus aut autem,False
1,1,2,quis ut nam facilis et officia qui,False
2,1,3,fugiat veniam minus,False
3,1,4,et porro tempora,True
4,1,5,laboriosam mollitia et enim quasi adipisci qui...,False
5,1,6,qui ullam ratione quibusdam voluptatem quia omnis,False
6,1,7,illo expedita consequatur quia in,False
7,1,8,quo adipisci enim quam ut ab,True
8,1,9,molestiae perspiciatis ipsa,False
9,1,10,illo est ratione doloremque quia maiores aut,True


In [91]:
todos_df.tail(10)

,userId,id,title,completed
190,10,191,temporibus atque distinctio omnis eius impedit...,True
191,10,192,ut quas possimus exercitationem sint voluptates,False
192,10,193,rerum debitis voluptatem qui eveniet tempora d...,True
193,10,194,sed ut vero sit molestiae,False
194,10,195,rerum ex veniam mollitia voluptatibus pariatur,True
195,10,196,consequuntur aut ut fugit similique,True
196,10,197,dignissimos quo nobis earum saepe,True
197,10,198,quis eius est sint explicabo,True
198,10,199,numquam repellendus a magnam,True
199,10,200,ipsam aperiam voluptates qui,False


In [93]:
todos_df2 = todos_df.reindex(todos_df.loc[:,'id'])
todos_df2.head()

,userId,id,title,completed
id,,,,
1,1.0,2.0,quis ut nam facilis et officia qui,False
2,1.0,3.0,fugiat veniam minus,False
3,1.0,4.0,et porro tempora,True
4,1.0,5.0,laboriosam mollitia et enim quasi adipisci qui...,False
5,1.0,6.0,qui ullam ratione quibusdam voluptatem quia omnis,False


That didn't do what I expected... I wanted to use the 'id' column as the index.

An alternative is to just drop the 'id' column since it has the same function as the df index.

Problem: reindex makes df conform to passed index, so e.g. we could resort in a particular way. In this case, the 'id' column starts at 1, so we just drop the first entry and most likely the last entry is null.

In [98]:
todos_df2.tail()

,userId,id,title,completed
id,,,,
196,10.0,197.0,dignissimos quo nobis earum saepe,True
197,10.0,198.0,quis eius est sint explicabo,True
198,10.0,199.0,numquam repellendus a magnam,True
199,10.0,200.0,ipsam aperiam voluptates qui,False
200,NaN,NaN,NaN,NaN


...this confirms what happened.

Let's try again... we just need to use `set_index`, which I didn't see in "Python for Data Analysis".

In [100]:
todos_df.set_index('id').head()

,userId,title,completed
id,,,
1,1,delectus aut autem,False
2,1,quis ut nam facilis et officia qui,False
3,1,fugiat veniam minus,False
4,1,et porro tempora,True
5,1,laboriosam mollitia et enim quasi adipisci qui...,False


That did what we want, so let's set `inplace=True` so that it mutates `todos_df`.

In [101]:
todos_df.set_index('id', inplace=True)
todos_df.head()

,userId,title,completed
id,,,
1,1,delectus aut autem,False
2,1,quis ut nam facilis et officia qui,False
3,1,fugiat veniam minus,False
4,1,et porro tempora,True
5,1,laboriosam mollitia et enim quasi adipisci qui...,False


### End pandas aside...

now we can test our DataFrame to SQL functions on `todos_df`

# More info on datatypes, entries

In [110]:
songs.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
0,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
0,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0


In [103]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71 entries, 0 to 0
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num_songs         71 non-null     int64  
 1   artist_id         71 non-null     object 
 2   artist_latitude   31 non-null     float64
 3   artist_longitude  31 non-null     float64
 4   artist_location   71 non-null     object 
 5   artist_name       71 non-null     object 
 6   song_id           71 non-null     object 
 7   title             71 non-null     object 
 8   duration          71 non-null     float64
 9   year              71 non-null     int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 6.1+ KB


In [104]:
logs = json_to_df(log_file_paths)
logs.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


In [106]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8056 entries, 0 to 357
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         6820 non-null   object 
 1   auth           8056 non-null   object 
 2   firstName      7770 non-null   object 
 3   gender         7770 non-null   object 
 4   itemInSession  8056 non-null   int64  
 5   lastName       7770 non-null   object 
 6   length         6820 non-null   float64
 7   level          8056 non-null   object 
 8   location       7770 non-null   object 
 9   method         8056 non-null   object 
 10  page           8056 non-null   object 
 11  registration   7770 non-null   float64
 12  sessionId      8056 non-null   int64  
 13  song           6820 non-null   object 
 14  status         8056 non-null   int64  
 15  ts             8056 non-null   int64  
 16  userAgent      7770 non-null   object 
 17  userId         8056 non-null   object 
dtypes: float6

In [108]:
logs.loc[:,'page'].value_counts()

NextSong            6820
Home                 806
Login                 92
Logout                90
Downgrade             60
Settings              56
Help                  47
About                 36
Upgrade               21
Save Settings         10
Error                  9
Submit Upgrade         8
Submit Downgrade       1
Name: page, dtype: int64

In [109]:
logs.loc[:,'level'].value_counts()

paid    6291
free    1765
Name: level, dtype: int64

In [123]:
def f(x):
    return len(str(x))
songs.loc[:,'artist_id'].map(f).value_counts()

18    71
Name: artist_id, dtype: int64

In [124]:
songs.loc[:,'song_id'].map(f).value_counts()

18    71
Name: song_id, dtype: int64

In [126]:
logs.loc[:, 'sessionId'].value_counts()

589    128
984    118
324    114
672    107
293    106
      ... 
587      1
234      1
125      1
230      1
6        1
Name: sessionId, Length: 941, dtype: int64